Code for TensorBoard
===

In [0]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = 'logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Code for Downloading data from Google Drive
===

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'13sCOigbP_BO5XdpbLn7o0jLQY3sDWZEs' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

Preparing the Dataset
===

In [0]:
import numpy as np

In [0]:
with open('./data/reviews.txt', 'r') as file:
  features = file.read()

In [0]:
features = features.split('\n')

## Getting trigrams

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
def vectorizer(text):
  vec = CountVectorizer(binary=True, min_df=3, ngram_range=(3, 3), token_pattern='(?u)\\b[a-z0-9\-\_]{3,}\\b', stop_words='english')
  vec.fit(text)
  return vec

In [0]:
vec = vectorizer(features)
ngrams = vec.transform(features).toarray()

In [0]:
ngrams

In [0]:
ngrams.shape

## One-hot encoding the labels

In [0]:
with open('./data/labels.txt', 'r') as file:
  labels = file.read()

In [0]:
labels = np.array([1 if label == 'positive' else 0 for label in labels.split('\n')])

In [0]:
from keras.utils import to_categorical

In [0]:
labels = to_categorical(labels)

## Splitting the dataset

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ngrams, labels[0:25000], test_size=0.30, stratify=labels[0:25000])

Building the Model
===

In [0]:
import tensorflow as tf

In [0]:
tf.reset_default_graph()

with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, ngrams.shape[1]],
                                 name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, labels.shape[1]],
                               name='input_labels')

  with tf.name_scope('model'):
    weights = tf.Variable(tf.random_normal(shape=[ngrams.shape[1], labels.shape[1]]),
                          name='weights')
    biases = tf.Variable(tf.random_normal(shape=[labels.shape[1]]))
    linear_model = tf.add(tf.matmul(input_features, weights), biases)
    predictions = tf.nn.softmax(linear_model)
    
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
      logits=linear_model, labels=input_labels))
  train_op = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)

with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1),
                                         tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)

summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='logs', graph=tf.get_default_graph())

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  
  # epochs = 10
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy, train_summary = sess.run([train_op, loss,
                                                               accuracy_op,
                                                              summary],
                                              feed_dict={input_features: batch_x,
                                                        input_labels: batch_y})
      writer.add_summary(summary=train_summary, global_step=mini_batch)
    print('Epoch {}, loss : {}, accuracy : {}'.format(epoch, train_loss,
                                                      train_accuracy))  
  
  test_accuracy = sess.run(accuracy_op, feed_dict={input_features: x_test, input_labels: y_test})
  print('Test accuracy : {}'.format(test_accuracy))

In [0]:
classes = ['Negative', 'Positive']
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: x_test[1].reshape(-1, x_test.shape[1])})
  print(prediction_)
  print('predicted class : {}, true class : {}'.format(sess.run(tf.argmax(prediction_, 1)), classes[sess.run(tf.argmax(y_test[1]))]))

In [0]:
features[100]

In [0]:
labels[100]

In [0]:
ngrams_example = vec.transform(['the food was extremely bad!']).toarray()

In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: ngrams_example.reshape(-1, ngrams_example.shape[1])})
  print(prediction_)
  print('predicted class : {}'.format(sess.run(tf.argmax(prediction_, 1))))